# **02456 - Project: Prediction-of-protein-isoforms-using-semi-supervised-learning**

This is the notebook associated with project *Prediction of protein isoforms using semi-supervised learning* by Lucas Brylle (s203832) and Nikolaj Hertz (s214644).

The data used is located on gbar dataserver on the path:

$\texttt{path\_to\_data = "/dtu-compute/datasets/iso\_02456/hdf5-row-sorted/"}$


The notebook is a concattenation of the files $\texttt{M1.py}$, $\texttt{M2.py}$, $\texttt{PCA.py}$ and $\texttt{FNN.py}$


----

The following packages are required for running the script

In [ ]:
# imports
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import time 
import os

import torch
from torch import nn, Tensor
import torch.optim as optim
import torch.nn.init as init
from torch.distributions import Normal, Distribution, Uniform

from collections import defaultdict


# dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from modules.MainSplit import MainSplit
from torch.utils.data import Subset
import torch.utils.data
import h5py
import random
import pickle


#from modules.helperFunctions import reduce, ReparameterizedDiagonalGaussian, init_dataloaders


from modules.plotting import *

from typing import *

from sklearn.decomposition import IncrementalPCA




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

-----

The following functions are used as helper functions

In [ ]:

def random_seed(random_seed):
    """
    Function to seed the data-split and backpropagation (to enforce reproducibility)
    """
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

class ReparameterizedDiagonalGaussian(Distribution):
    """
    A distribution `N(y | mu, sigma I)` compatible with the reparameterization trick given `epsilon ~ N(0, 1)`.
    """
    def __init__(self, mu: Tensor, log_sigma:Tensor):
        assert mu.shape == log_sigma.shape, f"Tensors `mu` : {mu.shape} and ` log_sigma` : {log_sigma.shape} must be of the same shape"
        self.mu = mu
        self.sigma = log_sigma.exp()

    def sample_epsilon(self) -> Tensor:
        """`\eps ~ N(0, I)`"""
        return torch.empty_like(self.mu).normal_().to(device)

    def sample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (without gradients)"""
        with torch.no_grad():
            return self.rsample()

    def rsample(self) -> Tensor:
        """sample `z ~ N(z | mu, sigma)` (with the reparameterization trick) """
        eps = torch.normal(0.0, 1.0, self.mu.shape).to(device)

        z = (self.mu + self.sigma * eps).to(device)
        return z

    def log_prob(self, z:Tensor) -> Tensor:
        """return the log probability: log `p(z)`"""
        m = Normal(self.mu, self.sigma)
        return m.log_prob(z).to(device)

def reduce(x:Tensor) -> Tensor:
    """for each datapoint: sum over all dimensions"""
    return x.view(x.size(0), -1).sum(dim=1)

def removeNaN(t1, t2):
    """
    function for removing unlabelled points
    """

    combined_nan_indices =  torch.sum(t1,axis=1)==0

    t1_masked = t1[torch.logical_not(combined_nan_indices)]
    t2_masked = t2[torch.logical_not(combined_nan_indices)]


    return t1_masked, t2_masked

def save_parameters(save_path, **kwargs):
    """
    Saving the parameters to a given path
    """
    with open(os.path.join(save_path,  "parameters.txt"), 'w') as f:  
        for key, value in kwargs.items():  
            f.write('%s:%s\n' % (key, value))


def init_folders(experiment_name):
    """
    Create folders for saving data.
    """
    main_path = os.getcwd()
    today = time.strftime("%d_%m")

    main_folder = os.path.join(main_path, 'results/')
    if not os.path.exists(main_folder):
        os.mkdir(main_folder)
    if not os.path.exists(main_folder + today):
        os.mkdir(main_folder + today)
    if not os.path.exists(main_folder + today + '/' + experiment_name):
        os.mkdir(main_folder + today + '/' + experiment_name)
    save_path = main_folder + today + '/' + experiment_name + '/'

    return save_path

------
# THE MODELS

The following will showcase the classes related to each model

## PCA - definition

Can be found within the training loop. Model is based upon sklearns Iterative PCA:

https://scikit-learn.org/stable/auto_examples/decomposition/plot_incremental_pca.html

## FNN - definition

In [ ]:

class FNN(nn.Module):
    def __init__(self, input_dim:int, N_isoform:int, p=0.5):
        
        super(FNN, self).__init__()

        # FNN
        self.regressor = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.Dropout(p=p),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.Dropout(p=p),
            nn.ReLU(),
            nn.Linear(1024, 2*N_isoform)
        )

        def init_weights(m):
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0)

        self.regressor.apply(init_weights)


    def forward(self, x: Tensor) -> Tensor:
        # this gave the nearly the same performance as a regular prediction. But it is more comparable with the regressor built into M2 now.
        mu, log_var = self.regressor(x).chunk(2, dim=-1)
        p_y = Normal(mu, log_var.exp())
        return p_y.rsample()

## M1 - definition

In [ ]:
class VariationalAutoencoder(nn.Module):
    """A Variational Autoencoder with
    * a Bernoulli observation model `p_\theta(x | z) = B(x | g_\theta(z))`
    * a Gaussian prior `p(z) = N(z | 0, I)`
    * a Gaussian posterior `q_\phi(z|x) = N(z | \mu(x), \sigma(x))`
    """

    def __init__(self, batch_cols,input_shape:torch.Size, latent_features:int) -> None:
        super(VariationalAutoencoder, self).__init__()
        self.input_shape = (batch_cols, )
        self.latent_features = latent_features
        self.observation_features =  batch_cols
        
        # Inference Network
        # Encode the observation `x` into the parameters of the posterior distribution
        # `q_\phi(z|x) = N(z | \mu(x), \sigma(x)), \mu(x),\log\sigma(x) = h_\phi(x)`
        self.encoder = nn.Sequential(
            nn.Linear(in_features=self.observation_features, out_features=256),
            #nn.BatchNorm1d(256), 
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            # A Gaussian is fully characterised by its mean \mu and variance \sigma**2
            nn.Linear(in_features=128, out_features=2*latent_features) # <- note the 2*latent_features
        )

        # Generative Model
        # Decode the latent sample `z` into the parameters of the observation model
        # `p_\theta(x | z) = \prod_i B(x_i | g_\theta(x))`
        self.decoder = nn.Sequential(
            nn.Linear(in_features=latent_features, out_features=128),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=2*self.observation_features),
        )

        # define the parameters of the prior, chosen as p(z) = N(0, I)
        self.register_buffer('prior_params', torch.zeros(torch.Size([1, 2*latent_features])))

        def init_weights(m):
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)

        self.encoder.apply(init_weights)
        self.decoder.apply(init_weights)

    def posterior(self, x:Tensor) -> Distribution:
        """return the distribution `q(x|x) = N(z | \mu(x), \sigma(x))`"""
        
        # compute the parameters of the posterior
        h_x = self.encoder(x)
        mu, log_sigma =  h_x.chunk(2, dim=-1)
        
        # return a distribution `q(x|x) = N(z | \mu(x), \sigma(x))`
        return ReparameterizedDiagonalGaussian(mu, log_sigma)

    def prior(self, batch_size:int=1)-> Distribution:
        """return the distribution `p(z)`"""
        prior_params = self.prior_params.expand(batch_size, *self.prior_params.shape[-1:])
        mu, log_sigma = prior_params.chunk(2, dim=-1)

        # return the distribution `p(z)`
        return ReparameterizedDiagonalGaussian(mu, log_sigma)

    def observation_model(self, z:Tensor) -> Distribution:
        """return the distribution `p(x|z)`"""
        mu, log_sigma = self.decoder(z).chunk(2, dim=-1)

        return Normal(mu, log_sigma, device)

    def forward(self, x) -> Dict[str, Any]:
        """compute the posterior q(z|x) (encoder), sample z~q(z|x) and return the distribution p(x|z) (decoder)"""
        
        # flatten the input
        x = x.view(x.size(0), -1)
        
        # define the posterior q(z|x) / encode x into q(z|x)
        qz = self.posterior(x)
        
        # define the prior p(z)
        pz = self.prior(batch_size=x.size(0))
        
        # sample the posterior using the reparameterization trick: z ~ q(z | x)
        z = qz.rsample()

        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)

        return {'px': px, 'pz': pz, 'qz': qz, 'z': z}


    def sample_from_prior(self, batch_size:int=100):
        """sample z~p(z) and return p(x|z)"""

        # degine the prior p(z)
        pz = self.prior(batch_size=batch_size)

        # sample the prior
        z = pz.rsample()

        # define the observation model p(x|z) = B(x | g(z))
        px = self.observation_model(z)

        return {'px': px, 'pz': pz, 'z': z}

class VariationalInference(nn.Module):
    def __init__(self, beta:float=1.):
        super().__init__()
        self.beta = beta

    def forward(self, model:nn.Module, x:Tensor) -> Tuple[Tensor, Dict]:

        # forward pass through the model
        
        outputs = model(x)
        
        # unpack outputs
        px, pz, qz, z = [outputs[k] for k in ["px", "pz", "qz", "z"]]
        
        # evaluate log probabilities
        log_px = reduce(px.log_prob(x))
        log_pz = reduce(pz.log_prob(z))
        log_qz = reduce(qz.log_prob(z))

        # compute the ELBO with and without the beta parameter:
        # `L^\beta = E_q [ log p(x|z) ] - \beta * D_KL(q(z|x) | p(z))`
        # where `D_KL(q(z|x) | p(z)) = log q(z|x) - log p(z)`
        kl = log_qz - log_pz
        elbo = log_px - kl # <- your code here
        beta_elbo = log_px - self.beta * kl

        # loss
        loss = -beta_elbo.mean()

        # prepare the output
        with torch.no_grad():
            diagnostics = {'elbo': elbo, 'log_px':log_px, 'kl': kl}

        return loss, diagnostics, outputs

## M2 - definition

In [ ]:
class M2(nn.Module):
    def __init__(self, 
                 latent_dim:int,
                 input_dim:int,
                 N_isoform:int,
                 alpha:float,
                 py_data:pd.DataFrame
                 ):
        
        super(M2, self).__init__()

        # this value is used in the loss function. It is defined to be 0.1 * N
        self.alpha = alpha

        self.py_data = py_data
        
        # q(z|x,y)
        # (it takes x and y as inputs)
        self.encoder = nn.Sequential(
            nn.Linear(in_features=input_dim + N_isoform, out_features=256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=2 * latent_dim) 
        )

        # p(x|z,y)
        # (it takes z and y as inputs)
        self.decoder = nn.Sequential( 
            nn.Linear(in_features=latent_dim + N_isoform, out_features=128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=2 * input_dim),
        )

        # Regression FNN
        # predicts y given x

        self.regressor = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),  
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 2*N_isoform)
        )

        # define the parameters of the prior, chosen as p(z) = N(0, I)
        self.register_buffer('prior_params', torch.zeros(torch.Size([1, 2*latent_dim])))



    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        # data manipulation
        y = y.to(device)
        x = x.to(device)

        # here it is checked which y's are not available
        # TODO: Make this a function
        index =  torch.sum(y,axis=1)==0

        # splits data in x and y.
        x_unlabelled = x[index]
        x_labelled = x[~index]

        y_labelled = y[~index]


        #* Calculate loss based on these splits
        # LABELLED LOS
        if torch.sum(torch.logical_not(index)) > 1:
            L = self.labelled_loss(x_labelled, y_labelled)
        else:
            L = torch.tensor(0).to(device)
            

        # UNLABELLED LOSS
        if torch.sum(index) > 1:
            U = self.unlabelled_loss(x_unlabelled)
        else:
            U = torch.tensor(0).to(device)
            

        J = L.sum() + U.sum()

        # TODO: write into function
        if torch.sum(torch.logical_not(index)) > 1:
            output_from_regressor = self.regressor(x_labelled)
            y_hat_mu, y_hat_log_sigma = output_from_regressor.chunk(2, dim=-1)

            q_y_x = Normal(y_hat_mu, y_hat_log_sigma.exp())

            log_q_y_x = -reduce(q_y_x.log_prob(y_labelled))
        else:
            log_q_y_x = torch.tensor(0.0).to(device)
        
        J_alpha = J  + self.alpha * log_q_y_x.mean()

        return J_alpha  

        
    def labelled_loss(self, x:Tensor, y:Tensor):
        #* Calculates the labelled loss, denoted L(x,y)
        # encoder now gets both the x and y!
        # TODO: Write this into a function
        input_to_encoder = torch.cat((x,y), dim=1)

        # output of encoder is mu, and log_var!
        output_from_encoder = self.encoder(input_to_encoder)
        # splits:
        mu_encoder, log_sigma_encoder =  output_from_encoder.chunk(2, dim=-1)

        # Now we can build a distribution over all the z's:
        q_z_xy = ReparameterizedDiagonalGaussian(mu_encoder, log_sigma_encoder)
        # (ps this stand for distribution of z given x and y:  q(z|x,y) )

        # then we need the prior over the y's:
        # TODO: insert the distribution over the y's. Should be accessed from py_data that contains the mu and sigma for the normal distribution. 
        #p_y = ReparameterizedDiagonalGaussian(py_data[0], sigma_p_y)
        
        p_y = Normal(torch.tensor(self.py_data['Mean'].values).to(device), 
                     torch.tensor(self.py_data["Standard_Deviation"].values).to(device))

        # PRIOR DISTRIBUTION OVER THE z's !!! They are the same as M1
        # TODO: Write this into a function
        prior_params = self.prior_params.expand(x.size(0), *self.prior_params.shape[-1:])
        mu_prior, log_sigma_prior = prior_params.chunk(2, dim=-1)
        p_z = ReparameterizedDiagonalGaussian(mu_prior, log_sigma_prior)


        # TO APPROXIMATE THE EXPECTATION, WE SAMPLE FROM q_z_xy (the expectation is over q_z_xy)
        z = q_z_xy.rsample()

        # DECODER
        # TODO: write this into a function
        input_to_decoder = torch.cat((z,y), dim=1)
        output_from_decoder = self.decoder(input_to_decoder)
        mu_decoder, log_sigma_decoder = output_from_decoder.chunk(2, dim=1)
        p_x_yz = Normal(mu_decoder, log_sigma_decoder.exp())



        #* AND FOR MY FINAL TRICK...
        # THis idea with reduce is stolen from the original VAE document. 
        # (it just sums up the probability across the non-batch dimension)
        log_p_x_yz = reduce(p_x_yz.log_prob(x))
        log_p_y =  reduce(p_y.log_prob(y))
        # print(log_p_y)
        log_p_z = reduce(p_z.log_prob(z))
        log_q_z_xy = reduce(q_z_xy.log_prob(z))



        L = (-1) * (log_p_x_yz + log_p_y + log_p_z - log_q_z_xy)
        # TODO: think about the minus
        return L


    def unlabelled_loss(self, x:Tensor):
        #* CALCULATES THE UNLABELLED LOSS, DENOTED U(x)

        # LETS FIND THE GAUSSIAN DISTRIBUTION OVER y
        output_from_regressor = self.regressor(x)
        y_hat_mu, y_hat_log_sigma = output_from_regressor.chunk(2, dim=-1)

        # the gaussian distribution over y's
        qy = Normal(y_hat_mu,y_hat_log_sigma.exp())

        # To approximate the expectation, we sample from over regression!!!
        y_hat = qy.sample()

        #* AND FOR MY FINAL TRICK... AGAIN:
        H = reduce(qy.entropy())

        U = (-1) * (-self.labelled_loss(x, y_hat) + H)
        return U

## Training the PCA-model

In [ ]:

# PARAMETERS (and the final parameters)
batch_size_ipca = 512
batch_size_nn = 128
n_components = 512 
n_epochs = 100
experiment_name = 'FINAL_PCA'
output_dim = 156958
learning_rate = 1e-4
random_seed_ = 1

############################################################################################################
#* INITIALIZATION
############################################################################################################


path_to_data = "/dtu-compute/datasets/iso_02456/hdf5-row-sorted/"

save_path = init_folders(experiment_name)

random_seed(random_seed_)

save_parameters(save_path,
                batch_size_ipca=batch_size_ipca,
                batch_size_nn=batch_size_nn,
                n_components=n_components,
                n_epochs=n_epochs,
                experiment_name=experiment_name,
                output_dim=output_dim,
                lr=learning_rate,
                random_seed_=random_seed_
                )


# IPCA will error if this is not uphold
assert n_components <= batch_size_ipca


############################################################################################################
#* THE DATA (MainSplit)
############################################################################################################
"""
In this datasplit the mentioned test-data in the report is also split in order to have labelled data to train the neural network, 
and to evaluate the neural network on unseen data (all in all having three splits (all stratified))

1. train_split: training the PCA
2. test_split: training the regressor
3. validate_split: evaluating the regressor
"""



# THE MAIN TRAINING SPLIT (some y's are 'labelled')
train_split = MainSplit(path_to_data, train=True)
train_loader = DataLoader(train_split, batch_size=batch_size_ipca, shuffle=True, drop_last=True)



# THE MAIN TEST SPLIT (All y's are 'labelled'), Is further split into a training for FNN (called test) and validation 
test_validation_split = MainSplit(path_to_data, train=False)


# splitting (stratified):
test_idx, validation_idx = train_test_split(np.arange(len(test_validation_split)),
                                            test_size=0.4,
                                            random_state=999,
                                            shuffle=True,
                                            stratify=test_validation_split.tissue_types)


test_dataset = Subset(test_validation_split, test_idx)
validation_dataset = Subset(test_validation_split, validation_idx)

# Testing splits
test_loader = DataLoader(test_dataset, batch_size=batch_size_nn, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size_nn, shuffle=True, drop_last=True)


############################################################################################################
#* THE REGRESSOR
############################################################################################################


#* Build the FFN
regressor = nn.Sequential(
            nn.Linear(n_components, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 156958)  # Output dimension for regression
        ).to(device)


############################################################################################################
#* TRAINING PCA
############################################################################################################


# define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(regressor.parameters(), lr=learning_rate)

# initialize the ipca model
ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size_ipca)

# save time
print("Fitting ipca model on train data...")
tic = time.time()

# fit the ipca model on the archs4 data
def train_pca(dataloader, model):
    for (x, y) in tqdm(dataloader, desc='PCA - Training'):
        model.partial_fit(x)

# for saving the pca model
from joblib import dump, load

train_pca(train_loader, ipca)

dump(ipca, save_path + '/ipca.joblib') 


# print time
toc = time.time()
print(f"Time to fit ipca model on train data: {toc-tic:.2f} seconds")



############################################################################################################
#* TRAIN THE REGRESSOR
############################################################################################################


def plot_temp(lists:dict, save_path):
    plt.figure()
    for n, v in lists.items():
        plt.plot(v, label='n')
    plt.legend()
    plt.savefig(save_path)
    plt.close()

def train_fnn(ipca,
            optimizer, 
            regressor, 
            train_data_loader,
            test_data_loader,
            num_epochs, 
            device,
            criterion):
    epoch = 0
    regressor.train()
    save_data = defaultdict(list)
    with tqdm(total=num_epochs * len(train_data_loader) * len(test_data_loader), desc='FNN - Training') as pbar:
        while epoch < num_epochs:
            epoch += 1
            epoch_data = defaultdict(list)
            
            for i, (x, y) in enumerate(train_data_loader):
                latent = ipca.transform(x)
                latent = torch.from_numpy(latent).to(device).float()

                y = y.to(device)

                y_pred = regressor(latent)
                loss = criterion(y_pred, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_data['fnn_trainloss'].append(loss.item())

                pbar.update(1)

                if i % 100:
                    tqdm.write("FNN_trainloss: " + str(loss.item()))


            
                
            for i, (x,y) in enumerate(test_data_loader):
                latent = ipca.transform(x)
                latent = torch.from_numpy(latent).to(device).float()

                y = y.to(device)

                y_pred = regressor(latent)
                loss = criterion(y_pred, y)
                optimizer.step()

                epoch_data['fnn_testloss'].append(loss.item())

                pbar.update(1)

                if i % 100:
                    tqdm.write("FNN_testloss: " + str(loss.item()))
    
            



            #plot_temp({'fnn_trainloss': epoch_data['fnn_trainloss']}, save_path + '/pca_fnn_trainloss' + str(epoch)+ '.png')
            #plot_temp({'fnn_testloss': epoch_data['fnn_testloss']}, save_path + '/pca_fnn_testloss' + str(epoch)+ '.png')
            save_data['fnn_trainloss'].append(np.mean(epoch_data['fnn_trainloss']))
            save_data['fnn_testloss'].append(np.mean(epoch_data['fnn_testloss']))


            np.save(save_path + 'pca_trainfnn.npy', save_data['fnn_trainloss'])
            np.save(save_path + 'pca_testfnn.npy', save_data['fnn_testloss'])

            np.save(save_path + 'fnntest_last_epoch.npy', epoch_data['fnn_testloss'])

            torch.save(regressor.state_dict(), save_path + '/regressor.pth')





criterion = nn.MSELoss()

save_fnn = train_fnn(ipca, 
                    optimizer, 
                    regressor, 
                    test_loader,
                    validation_loader, 
                    n_epochs, 
                    device, 
                    criterion)

## Training the FNN-model

In [ ]:
# Saving parameters of the run (this is the final run)

path_to_data = "/dtu-compute/datasets/iso_02456/hdf5-row-sorted/"
input_dim = 18965
N_isoform = 156958
num_epochs = 100
random_seed_ = 1
batch_size = 64
learning_rate = 1e-4
experiment_name = 'FNN_final'


############################################################################################################
#* INITIALIZATION
############################################################################################################


save_path = init_folders(experiment_name)


save_parameters(save_path, 
                experiment_name=experiment_name,
                input_dim=input_dim, 
                N_isoform=N_isoform, 
                num_epochs=num_epochs, 
                random_seed_=random_seed_, 
                batch_size=batch_size,
                learning_rate=learning_rate)


random_seed(random_seed_)


############################################################################################################
#* THE DATA (MainSplit)
############################################################################################################
"""
this method can't use the unlabelled data
"""


# THE MAIN TEST SPLIT (All y's are 'labelled'), Is further split into a training for FNN (called test) and validation 
test_validation_split = MainSplit(path_to_data, train=False)


# splitting (stratified):
test_idx, validation_idx = train_test_split(np.arange(len(test_validation_split)),
                                            test_size=0.4,
                                            random_state=999,
                                            shuffle=True,
                                            stratify=test_validation_split.tissue_types)


test_dataset = Subset(test_validation_split, test_idx)
validation_dataset = Subset(test_validation_split, validation_idx)


train_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


############################################################################################################
#* TRAINING THE MODEL
############################################################################################################



model = FNN(input_dim, N_isoform)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#, weight_decay=1e-4)

# Training params



def train(model, optimizer, num_epochs, criterion = nn.MSELoss()):
    epoch = 0
    train_data = defaultdict(list)
    validation_data = defaultdict(list)
    
    
    while epoch < num_epochs:
        epoch += 1
        training_epoch_data = defaultdict(list)
        validation_epoch_data = defaultdict(list)

        model.train()
        for (x, y) in tqdm(train_loader, desc = "FNN - Training"):
            y = y.to(device)
            x = x.to(device)

            optimizer.zero_grad()

            y_pred = model(x)

            loss = criterion(y_pred, y)

            training_epoch_data["FNN"].append(loss.item())

            loss.backward()

            clipping_value = 0.5 # arbitrary value of your choosing
            torch.nn.utils.clip_grad_norm_(model.parameters(), clipping_value)

            optimizer.step()
            
            training_epoch_data["loss"].append(loss.item())
            

        train_FNN = np.mean(training_epoch_data["FNN"])
        train_data["FNN"].append(train_FNN)

        with torch.no_grad():
            model.eval()

            for x, y in tqdm(validation_loader, desc = "FNN - Validation"):
                y = y.to(device)
                x = x.to(device)

                y_pred = model(x)

                loss = criterion(y_pred, y)

                validation_epoch_data["FNN"].append(loss.item())


            validation_FNN = np.mean(validation_epoch_data["FNN"])
            validation_data["FNN"].append(validation_FNN)

        np.save(save_path + '/training_FNN.npy', train_data['FNN'])
        np.save(save_path + '/validation_FNN.npy', validation_data['FNN'])
        np.save(save_path + '/last_epoch_FNN.npy', validation_epoch_data["FNN"])

        # a plot function that is not so important
        #createLossPlotFNN2(train_data['FNN'], validation_data['FNN'], 'LONELY_FNN', save_path)

train(model, optimizer, num_epochs)    

## Training the M1-model

In [ ]:


# parameters for the final run
batch_size = 256
latent_features = 512
num_epochs = 100
experiment_name = 'm1_final_run'
beta = 1.0
output_dim = 156958
lr = 3e-4
input_dim = 18965
weight_decay = 1e-4
random_seed_ = 1
clipping_value = 0.5


############################################################################################################
#* INITIALIZATION
############################################################################################################


# seeding 
random_seed(random_seed_)

# save_folders
save_path = init_folders(experiment_name)

save_parameters(save_path,
                batch_size=batch_size,
                latent_features=latent_features,
                num_epochs=num_epochs,
                experiment_name=experiment_name,
                beta=beta,
                output_dim=output_dim,
                lr=lr,
                input_dim=input_dim,
                random_seed_=random_seed_,
                weight_decay=weight_decay,
                clipping_value=clipping_value,
                )


# Regression FNN
regressor = nn.Sequential(
            nn.Linear(latent_features, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim)  # Output dimension for regression
        ).to(device)


############################################################################################################
#* THE DATA (MainSplit)
############################################################################################################
"""
1. train_split: training the unsupervised VAE
2. test_split: evaluate the unsupervised VAE AND training the regressor
3. validate_split: evaluating the regressor
"""


path_to_data = "/dtu-compute/datasets/iso_02456/hdf5-row-sorted/"



from torch.utils.data import DataLoader
from modules.MainSplit import MainSplit
from torch.utils.data import Subset


# THE MAIN TRAINING SPLIT (some y's are 'labelled')
train_split = MainSplit(path_to_data, train=True)
train_loader = DataLoader(train_split, batch_size=batch_size, shuffle=True)



# THE MAIN TEST SPLIT (All y's are 'labelled'), Is further split into a training for FNN (called test) and validation 
test_validation_split = MainSplit(path_to_data, train=False)


# splitting (stratified):
test_idx, validation_idx = train_test_split(np.arange(len(test_validation_split)),
                                            test_size=0.4,
                                            random_state=999,
                                            shuffle=True,
                                            stratify=test_validation_split.tissue_types)


test_dataset = Subset(test_validation_split, test_idx)
validation_dataset = Subset(test_validation_split, validation_idx)

# Testing splits
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


############################################################################################################
#* TRAINING THE MODEL
############################################################################################################


# VAE
vae = VariationalAutoencoder(input_dim, batch_size, latent_features)

# Evaluator: Variational Inference

vi = VariationalInference(beta=beta)

# FNN definition

criterion = nn.MSELoss()

# The Adam optimizer works really well with VAEs.
optimizerVAE = torch.optim.Adam(vae.parameters(), lr=lr)
optimizerFNN = torch.optim.Adam(regressor.parameters(), lr=lr)

# define dictionary to store the training curves
training_data = defaultdict(list)
validation_data = defaultdict(list)

# move the model to the device
vae = vae.to(device)
vi = vi.to(device)



save_data = defaultdict(list)


def train_vae(vae, 
            vi, 
            train_data_loader, 
            test_data_loader,
            optimizer, 
            num_epochs, 
            device):
    save_data = defaultdict(list)

    epoch = 0
    vae.train()
    with tqdm(total=num_epochs * (len(train_data_loader) + len(test_data_loader)), desc='VAE - Training') as pbar:
        while epoch < num_epochs:
            epoch += 1
            epoch_data = defaultdict(list)
            
            
            
            vae.train()
            for i, (x, _ ) in enumerate(train_data_loader):
                x = x.to(device)
                #y = y.to(device)

                # Forward pass through VAE and obtain VAE loss
                vae_loss, diagnostics, outputs = vi(vae, x)
                
                optimizer.zero_grad()
                vae_loss.backward()

                torch.nn.utils.clip_grad_norm_(vae.parameters(), clipping_value)


                optimizer.step()

                epoch_data['vae_trainloss'].append(vae_loss.item())

                pbar.update(1)

                if i % 100 == 0:
                    tqdm.write("VAE_trainloss: " + str(vae_loss.item()))
            
            vae.eval()
            for i, (x, _) in enumerate(test_data_loader):
                x = x.to(device)

                vae_loss, diagnostics, outputs = vi(vae, x)

                epoch_data['vae_testloss'].append(vae_loss.item())

                

            
            #plot_temp({'vae_loss': epoch_data['vae_loss']}, save_path + '/m1_loss' + str(epoch) + '.png')
            save_data['vae_trainloss'].append(np.mean(epoch_data['vae_trainloss']))
            save_data['vae_testloss'].append(np.mean(epoch_data['vae_testloss']))


            np.save(save_path + '/m1_trainloss.npy', save_data['vae_trainloss'])
            np.save(save_path + '/m1_testloss.npy', save_data['vae_testloss'])

            torch.save(vae.encoder.state_dict(), save_path + '/encoder.pth')
            #torch.save(vae.decoder.state_dict(), save_path + '/decoder.pth')


    

    
def train_fnn(vae,
            vi,
            optimizer, 
            regressor, 
            train_data_loader,
            test_data_loader,
            num_epochs, 
            device,
            criterion):
    epoch = 0
    vae.train()
    
    save_data = defaultdict(list)
    with tqdm(total=num_epochs * (len(train_data_loader) + len(test_data_loader)), desc='FNN - Training') as pbar:
        while epoch < num_epochs:
            epoch += 1
            epoch_data = defaultdict(list)
            regressor.train()
            for i, (x, y) in enumerate(train_data_loader):
                x = x.to(device)
                y = y.to(device)

                latent = vae(x)['z']


                y_pred = regressor(latent)

                loss = criterion(y_pred, y)


                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_data['fnn_trainloss'].append(loss.item())

                pbar.update(1)

                if i % 100:
                    tqdm.write("FNN_trainloss: " + str(loss.item()))
            


            
            regressor.eval()
            for i, (x,y) in enumerate(test_data_loader):
                
                x = x.to(device)
                y = y.to(device)


                latent = vae(x)['z']   
                y_pred = regressor(latent)



                loss = criterion(y_pred, y)
                optimizer.step()

                epoch_data['fnn_testloss'].append(loss.item())

                pbar.update(1)

                if i % 100:
                    tqdm.write("FNN_testloss: " + str(loss.item()))

        

            save_data['fnn_trainloss'].append(np.mean(epoch_data['fnn_trainloss']))
            save_data['fnn_testloss'].append(np.mean(epoch_data['fnn_testloss']))
            
            np.save(save_path + 'fnntest_last_epoch.npy', epoch_data['fnn_testloss'])

            np.save(save_path + 'm1_trainfnn.npy', save_data['fnn_trainloss'])
            np.save(save_path + 'm1_testfnn.npy', save_data['fnn_testloss'])

            torch.save(regressor.state_dict(), save_path + '/regressor.pth')






train_vae(vae, 
        vi, 
        train_loader, 
        test_loader,
        optimizerVAE, 
        num_epochs, 
        device)


train_fnn(vae,
        vi,
        optimizerFNN, 
        regressor, 
        test_loader,
        validation_loader, 
        num_epochs, 
        device, 
        criterion)

## Training the M2-model

In [ ]:

# parameters
random_seed_ = 1
batch_size = 256
N_isoform = 156958
latent_dim = 256
input_dim = 18965

num_epochs = 100
clipping_value = 0.5
N_unlabelled = 167884
learning_rate = 1e-4
weight_decay = 1e-4

# Seeding 
random_seed(random_seed_)


############################################################################################################
#* THE DATA (MainSplit and the pre-calculated Gaussian parameters for p(y))
############################################################################################################
"""
1. train_split: train the M2
2. test_split: evaluate the M2 
"""


path_to_data = "/dtu-compute/datasets/iso_02456/hdf5-row-sorted/"



# this should be the path to the means and sd of the y's
csv_path = '/zhome/31/1/155455/DeepLearningProject23/GaussianIsoforms.csv'

py_csv = pd.read_csv(csv_path)

# THE MAIN TRAINING SPLIT (some y's are 'labelled')
train_split = MainSplit(path_to_data, train=True)
train_loader = DataLoader(train_split, batch_size=batch_size, shuffle=True)

# as stated in the KINGMA ARTICLE
alpha = 0.1 * len(train_split)

# THE MAIN TEST SPLIT (All y's are 'labelled'), Is further split into a training for FNN (called test) and validation 
test_validation_split = MainSplit(path_to_data, train=False)
test_loader = DataLoader(test_validation_split, batch_size=batch_size, shuffle=True, drop_last=True)


############################################################################################################
#* TRAINING THE MODEL
############################################################################################################


# define dictionary to store the training curves
training_data = defaultdict(list)
validation_data = defaultdict(list)

# Model
vae = M2(latent_dim, input_dim, N_isoform, alpha=alpha, py_data = py_csv).to(device)

# Optimizer
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Loss function
criterion = nn.MSELoss()

# Making sure that things are on CUDA
vae = vae.to(device)

def train(vae, optimizer, num_epochs):
    epoch = 0
    train_data = defaultdict(list)
    validation_data = defaultdict(list)

    while epoch < num_epochs:
        epoch += 1
        training_epoch_data = defaultdict(list)
        validation_epoch_data = defaultdict(list)
        i = 0
        vae.train()
        for x, y in tqdm(train_loader, desc = "VAE - Training"):
            
            y = y.to(device)
            x = x.to(device)

            optimizer.zero_grad()
            loss = vae(x, y)

            y_hat_mu, y_hat_log_sigma = vae.regressor(x).chunk(2, dim=-1)

            # the gaussian distribution over y's
            qy = Normal(y_hat_mu, y_hat_log_sigma.exp())

            # To approximate the expectation, we sample from over regression!!!
            y_pred = qy.sample()

            y_loss, y_pred_loss = removeNaN(y, y_pred)

            mse_loss = criterion(y_pred_loss, y_loss)

            training_epoch_data["FNN"].append(mse_loss.item())

            loss.backward()

                # arbitrary value of your choosing
            torch.nn.utils.clip_grad_norm_(vae.parameters(), clipping_value)

            optimizer.step()
            
            training_epoch_data["loss"].append(loss.item())

            if i % 200:
                #tqdm.write(str(loss.item()))
                #tqdm.write(str(mse_loss.item()))
                print(str(np.mean(training_epoch_data["loss"][-100:])))
                print("number of points:" + str(y_loss.size(0)) + " loss: " + str(np.mean(training_epoch_data["FNN"][-100:])))

            i += 1
            
        train_loss = np.mean(training_epoch_data["loss"])
        train_data["loss"].append(train_loss)

        train_FNN = np.mean(training_epoch_data["FNN"])
        train_data["FNN"].append(train_FNN)

        with torch.no_grad():
            vae.eval()
            i = 0

            for x, y in tqdm(test_loader, desc = "VAE - Validation"):
                y = y.to(device)
                x = x.to(device)

                loss = vae(x,y)

                y_hat_mu, y_hat_log_sigma = vae.regressor(x.to(device)).chunk(2, dim=-1)

                # the gaussian distribution over y's
                qy = Normal(y_hat_mu, y_hat_log_sigma.exp())

                # To approximate the expectation, we sample from over regression!!!
                y_pred = qy.sample()

                y_l, y_pred_l = removeNaN(y, y_pred)

                mse_loss = criterion(y_pred_l, y_l)

                validation_epoch_data["FNN"].append(mse_loss.item())
                validation_epoch_data["loss"].append(loss.item())
                if i % 100:
                    print(str(loss.item()))
                    print(str(mse_loss.item()))

                i += 1
            
            validation_loss = np.mean(validation_epoch_data["loss"])
            validation_data["loss"].append(validation_loss)

            validation_FNN = np.mean(validation_epoch_data["FNN"])
            validation_data["FNN"].append(validation_FNN)

        with torch.no_grad():    
            createLossPlotFNN(train_data["loss"], validation_data["loss"], "leg")
            createLossPlotFNN(train_data["FNN"], validation_data["FNN"], "leg2")

    # Saving model and epoch results
    # with open('GridCheck/train_data3' + '.pkl', 'wb') as fp:
    #     pickle.dump(train_data, fp)
    # with open('GridCheck/validation_data3' + '.pkl', 'wb') as fp:
    #     pickle.dump(validation_data, fp)

    save_path = "GridCheck/" 
    torch.save(vae.encoder.state_dict(),  save_path + "encoder3" + ".pth")
    torch.save(vae.decoder.state_dict(),  save_path + "decoder3" + ".pth")
    torch.save(vae.regressor.state_dict(),save_path + "regressor3"+".pth")


train(vae, optimizer, num_epochs)


------

This should generate the relevant files found in the epoch_data. By then using the script plot_mse, plot_elbo and R2_calculation it is possible to recreate the results from our report.